In [1]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path))

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
import vectorian

from vectorian.importers import NovelImporter
from vectorian.embeddings import FastText
from vectorian.session import LabSession

embedding = FastText("en")

im = NovelImporter(nlp)
doc = im("/Users/arbeit/A Child's Dream of a Star.txt")

session = LabSession(
    [doc],
    [embedding])

Matching checksum for /Users/arbeit/Projects/vectorian-2021/vectorian/core/cpp/core.cpp --> not compiling


100%|██████████| 58/58 [00:01<00:00, 43.55it/s]


In [10]:
query = nlp("literal")
session.find(query, n=3)

FloatProgress(value=0.0, layout=Layout(width='100%'), max=1.0)